In [1]:
from pandas_datareader import data as pdr
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import cvxpy as cvx
from scipy.stats import norm
from numpy.linalg import inv
import math
import random
import benchmarks

# for plotting
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import matplotlib.pyplot as plt

from benchmarks import *

# Initialize plotly offline
plotly.offline.init_notebook_mode(connected=True)

## Load Data

### Hundred stocks

In [2]:
# Read in the data
hundred_stock_prices = pd.read_csv('./00_Data/hundred_stocks.csv', header=0, index_col = 0)

In [3]:
hundred_stock_prices.index = pd.to_datetime(hundred_stock_prices.index)

#Process Stock Data
# hundred_stock_relative_prices, hundred_stock_returns = process_stock_data(hundred_stock_prices)

#Read processed data to save time
hundred_stock_relative_prices = pd.read_csv('./00_Data/hundred_stock_relative_prices.csv', header=0, index_col = 0)
hundred_stock_returns = pd.read_csv('./00_Data/hundred_stock_returns.csv', header=0, index_col = 0)

#### SPY

In [4]:
# download_stock_data('2001-01-02', '2019-07-17', ['SPY'], 'SPY_same_dates_as_hundred_stocks')

In [5]:
# Read in the data
SPY_prices = pd.read_csv('./00_Data/SPY_same_dates_as_hundred_stocks.csv', header=0, index_col = 0)
SPY_prices.index = pd.to_datetime(hundred_stock_prices.index)

#Process Stock Data
# SPY_relative_prices, SPY_returns = process_stock_data(SPY_prices)

# Read in data to save time
SPY_relative_prices = pd.read_csv('./00_Data/SPY_relative_prices_2001to2019.csv', header=0, index_col = 0)
SPY_returns = pd.read_csv('./00_Data/SPY_returns_2001to2019.csv', header=0, index_col = 0)

## Risk Aware Portfolio Selection Algorithm
#### Bug: cannot initialize gamma as cp.Parameter

In [6]:
def risk_aware_portfolio(stock_returns, gamma, delta):
    """
    input:
    stock_return: returns for the stock data
    gamma: the confidence level
    delta: number of dates we use to estimate the density of returns
    
    output:
    portfolio: portfolio selected
    """
    u = cvx.Variable((stock_returns.shape[1], 1))
    a = cvx.Variable()
    #     gamma = cp.Parameter(gamma)

    constraints = [cvx.sum(u) == 1, u >= 0]

    portfolio = []

    for t in range(2, len(stock_returns)):
        coefficient = 1 / ((delta + t - 1) * (1 - gamma))

        sliced = stock_returns.iloc[:delta]
        sliced = sliced.to_numpy()
        inner_part = cvx.pos(sliced * (-u) - a)
        first_sum = cvx.sum(inner_part)

        sliced2 = stock_returns.iloc[:t - 1]
        sliced2 = sliced2.to_numpy()
        inner_part2 = sliced2 * (-u) - a
        second_sum = cvx.sum(cvx.pos(inner_part2))

        obj = cvx.Minimize(a + coefficient * (first_sum + second_sum))

        prob = cvx.Problem(obj, constraints)
        prob.solve(solver='SCS')
        portfolio.append(u.value)

    return pd.DataFrame(np.array(portfolio).reshape(len(portfolio), len(portfolio[0])))

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 70 days here

In [7]:
hundred_stock_returns_wo_0 = hundred_stock_returns.replace(0, 0.001)

In [8]:
risk_aware_selection = risk_aware_portfolio(np.log(hundred_stock_returns_wo_0.iloc[:70]), 0.95, 30)

In [9]:
benchmark_portfolio(risk_aware_selection.T,
                    'Risk-aware',
                    SPY_relative_prices.iloc[:68], # two less days since beginning from 2
                    stock_returns=hundred_stock_returns.iloc[:68],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:68])

## Bandit Algorithm

## Portfolio selection with original UCB

In [10]:
def ucb_bandit_portfolio(stock_returns):
    n_time, n_assets = stock_returns.shape # n_time >> n_assets in our case
    portfolio = np.zeros((n_assets, n_time)) # the output portfolio
    Rbar = np.zeros((n_assets,1)) # empirical mean of return for assets

    num_selected = {}
    for i in range(n_assets):
        num_selected[i] = 0
        
    # for loop up until n_time, output the bandit portfolio at each time t
    
    for t in range(n_time):
        
        if t < n_assets:
            portfolio[t, t] = 1
            Rbar[t] = stock_returns.iloc[t, t]
            num_selected[t] = 1
            continue
            
        max_asset = 0
        max_upper_bound = 0
        for asset in range(n_assets):
            
            avg_reward = Rbar[asset]
            right_part = np.sqrt(2*np.log(t) / num_selected[asset])
            upper_bound = avg_reward + right_part
            
            if upper_bound > max_upper_bound:
                max_asset = asset
                max_upper_bound = upper_bound
                
        portfolio[max_asset, t] = 1
        #pull
        Rbar[max_asset] = (Rbar[max_asset] * num_selected[max_asset] + stock_returns.iloc[t, max_asset]) / (num_selected[max_asset] + 1)
        num_selected[max_asset] += 1
        
    return portfolio

In [11]:
og_ucb_selection = ucb_bandit_portfolio(hundred_stock_returns)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 70 days here

In [12]:
benchmark_portfolio(og_ucb_selection.T[:70],
                    'Original UCB',
                    SPY_relative_prices.iloc[:70],
                    stock_returns=hundred_stock_returns.iloc[:70],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:70])

## EXP3

In [13]:
def distr(weights, gamma=0.0):
    theSum = float(sum(weights))
    return tuple((1.0 - gamma) * (w / theSum) + (gamma / len(weights)) for w in weights)

In [14]:
def draw(weights):
    choice = random.uniform(0, sum(weights))
    choiceIndex = 0

    for weight in weights:
        choice -= weight
        if choice <= 0:
            return choiceIndex

        choiceIndex += 1

In [15]:
def exp3(stock_returns, gamma = 0.0):
    n_time, n_assets = stock_returns.shape # n_time >> n_assets in our case
    portfolio = np.zeros((n_assets, n_time))
    weights = [1] * n_assets
    
    for t in range(n_time):
        
        probabilityDistribution = distr(weights, gamma)
        choice = draw(probabilityDistribution)
        
        theReward = stock_returns.iloc[t, choice]

        estimatedReward = 1.0 * theReward / probabilityDistribution[choice]
        weights[choice] *= math.exp(
            estimatedReward * gamma /
            n_assets)  # important that we use estimated reward here!
        
        portfolio[choice, t] = 1

    return portfolio

In [16]:
exp3_selection = exp3(hundred_stock_returns, 0.8)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 100 days

In [17]:
benchmark_portfolio(exp3_selection.T[:70],
                    'EXP3',
                    SPY_relative_prices.iloc[:70],
                    stock_returns=hundred_stock_returns.iloc[:70],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:70])

## Improved EXP3

In [18]:
def improved_exp3(stock_returns, gamma = 0.0):
    n_time, n_assets = stock_returns.shape # n_time >> n_assets in our case
    portfolio = np.zeros((n_assets, n_time))
    weights = [1] * n_assets
    
    for t in range(n_time):
        
        probabilityDistribution = distr(weights, gamma)
        choice = draw(probabilityDistribution)
        
        theReward = stock_returns.iloc[t, choice]

        rewardMin, rewardMax = -1, 1
        
        for choice in range(n_assets):
            rewardForUpdate = stock_returns.iloc[t, choice]
            scaledReward = (rewardForUpdate - rewardMin) / (rewardMax - rewardMin)
            estimatedReward = 1.0 * scaledReward / probabilityDistribution[choice]
            weights[choice] *= math.exp(estimatedReward * gamma / n_assets)
            
#         estimatedReward = 1.0 * theReward / probabilityDistribution[choice]
#         weights[choice] *= math.exp(
#             estimatedReward * gamma /
#             n_assets)  # important that we use estimated reward here!
        
        portfolio[choice, t] = 1

    return portfolio
    

In [19]:
improved_exp3_selection = improved_exp3(hundred_stock_returns)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 100 days

In [20]:
benchmark_portfolio(improved_exp3_selection.T[:70],
                    'Improved EXP3',
                    SPY_relative_prices.iloc[:70],
                    stock_returns=hundred_stock_returns.iloc[:70],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:70])

## Improved UCB

In [21]:
def improved_ucb(stock_returns):
    n_time, n_assets = stock_returns.shape  # n_time >> n_assets in our case
    portfolio = np.zeros((n_assets, n_time))  # the output portfolio
    Rbar = np.zeros((n_assets, 1))  # empirical mean of return for assets

    num_selected = {}
    for i in range(n_assets):
        num_selected[i] = 0

    # for loop up until n_time, output the bandit portfolio at each time t

    for t in range(n_time):

        if t < n_assets:
            portfolio[t, t] = 1
            Rbar[t] = stock_returns.iloc[t, t]
            num_selected[t] = 1
            continue

        max_asset = 0
        max_upper_bound = 0

        for asset in range(n_assets):

            avg_reward = Rbar[asset]
            right_part = np.sqrt(2 * np.log(t) / num_selected[asset])
            upper_bound = avg_reward + right_part

            if upper_bound > max_upper_bound:
                max_asset = asset
                max_upper_bound = upper_bound
            
            # Update all UCBs, which includes the max asset.
            Rbar[asset] = (Rbar[asset] * num_selected[asset] +
                               stock_returns.iloc[t, asset]) / (
                                   num_selected[asset] + 1)

        portfolio[max_asset, t] = 1
        num_selected[max_asset] += 1

    return portfolio

In [22]:
improved_ucb_selection = improved_ucb(hundred_stock_returns)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 100 days

In [23]:
benchmark_portfolio(improved_ucb_selection.T[:70],
                    'Improved UCB',
                    SPY_relative_prices.iloc[:70],
                    stock_returns=hundred_stock_returns.iloc[:70],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:70])

## UCB + Risk aware

Note that default gamma is set to 0.5

In [24]:
def ucb_plus_riskaware(stock_returns, lambd, gamma=0.5, delta=30):
    risk_aware_selection = risk_aware_portfolio(stock_returns, gamma,
                                                delta).values
    ucb_selection = ucb_bandit_portfolio(stock_returns).T[2:] # drop first two to match risk aware's dates
    return lambd * ucb_selection + (1 - lambd) * risk_aware_selection

In [25]:
ucb_plus_riskaware_selection = ucb_plus_riskaware(
    hundred_stock_returns.iloc[:70], 0.5)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 70 days

In [26]:
benchmark_portfolio(ucb_plus_riskaware_selection,
                    'UCB + risk aware',
                    SPY_relative_prices.iloc[:68],
                    stock_returns=hundred_stock_returns.iloc[:68],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:68])

## EXP3 + Risk aware


Note that default gamma is set to 0.5

In [27]:
def exp3_plus_riskaware(stock_returns, lambd, gamma=0.5, delta=30):
    risk_aware_selection = risk_aware_portfolio(stock_returns, gamma,
                                                delta).values
    exp3_selection = exp3(stock_returns).T[
        2:]  # drop first two to match risk aware's dates
    return lambd * exp3_selection + (1 - lambd) * risk_aware_selection

In [28]:
exp3_plus_riskaware_selection = exp3_plus_riskaware(
    hundred_stock_returns.iloc[:70], 0.5)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 100 days

In [29]:
benchmark_portfolio(exp3_plus_riskaware_selection,
                    'exp3 + risk aware',
                    SPY_relative_prices.iloc[:68],
                    stock_returns=hundred_stock_returns.iloc[:68],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:68])

## Improved UCB + Risk aware

Note that default gamma is set to 0.5

In [30]:
risk_aware_selection = risk_aware_portfolio(hundred_stock_returns.iloc[:70], 0.5,
                                                30).values 

In [31]:
improved_ucb_selection = improved_ucb(hundred_stock_returns.iloc[:70]).T[
        2:]  # drop first two to match risk aware's dates

In [32]:
x = pd.DataFrame(0.5 * improved_ucb_selection + 0.5 * risk_aware_selection).values

In [33]:
def improved_ucb_plus_riskaware(stock_returns, lambd, gamma=0.5, delta=30):
    risk_aware_selection = risk_aware_portfolio(stock_returns, gamma,
                                                delta).values
    improved_ucb_selection = improved_ucb(stock_returns).T[
        2:]  # drop first two to match risk aware's dates
    return lambd * improved_ucb_selection + (1 - lambd) * risk_aware_selection

In [34]:
improved_ucb_plus_riskaware_selection = improved_ucb_plus_riskaware(
    hundred_stock_returns.iloc[:70], 0.5)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 100 days

In [35]:
benchmark_portfolio(improved_ucb_plus_riskaware_selection,
                    'improved ucb + risk aware',
                    SPY_relative_prices.iloc[:68],
                    stock_returns=hundred_stock_returns.iloc[:68],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:68])

## Improved EXP3 + Risk aware

Note that default gamma is set to 0.5

In [36]:
def improved_exp3_plus_riskaware(stock_returns, lambd, gamma=0.5, delta=30):
    risk_aware_selection = risk_aware_portfolio(stock_returns, gamma,
                                                delta).values
    improved_exp3_selection = improved_exp3(stock_returns).T[2:]  # drop first two to match risk aware's dates
    return lambd * improved_exp3_selection + (1 - lambd) * risk_aware_selection

In [37]:
improved_exp3_plus_riskaware_selection = improved_exp3_plus_riskaware(
    hundred_stock_returns.iloc[:70], 0.5)

### Benchmarking
> **WARNING**: Takes long time to run! Only using first 100 days

In [38]:
benchmark_portfolio(improved_exp3_plus_riskaware_selection,
                    'improved exp3 + risk aware',
                    SPY_relative_prices.iloc[:68],
                    stock_returns=hundred_stock_returns.iloc[:68],
                    stock_prices_norm=hundred_stock_relative_prices.iloc[:68])